In [ ]:
import matplotlib.pyplot as plt

import pandas as pd

import numpy as np

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense

from tensorflow.keras.layers import LSTM

from tensorflow.keras.layers import Dropout

from tensorflow.keras.layers import *

from tensorflow.keras.callbacks import EarlyStopping

from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.metrics import mean_squared_error

from sklearn.metrics import mean_absolute_percentage_error

from sklearn.model_selection import train_test_split

from sklearn.model_selection import TimeSeriesSplit

import matplotlib.pyplot as plt

,open,high,low,close,adj_close,volume
date,,,,,,
NaN,NFLX,NFLX,NFLX,NFLX,NFLX,NFLX
2002-05-23,1.1564290523529053,1.2428569793701172,1.1457140445709229,1.1964290142059326,1.1964290142059326,104790000
2002-05-24,1.214285969734192,1.225000023841858,1.1971429586410522,1.2100000381469727,1.2100000381469727,11104800
2002-05-28,1.2135709524154663,1.2321430444717407,1.157142996788025,1.157142996788025,1.157142996788025,6609400
2002-05-29,1.1642860174179077,1.1642860174179077,1.0857139825820923,1.1035710573196411,1.1035710573196411,6757800


In [ ]:
import pandas as pd
stock_data = pd.read_csv('data/Netflix_Dataset.csv', index_col='date') 
stock_data.head()

In [ ]:
import matplotlib.pyplot as plt

# Assuming 'stock_data' has a 'Date' column or is using Date as the index

plt.figure(figsize=(10, 6))

plt.plot(stock_data['High'], label='High', linewidth=2)

plt.plot(stock_data['Low'], label='Low', linewidth=1)

# Adding labels and title

plt.title('Stock High and Low Prices')

plt.xlabel('Date')

plt.ylabel('Price')

plt.legend()

# Rotating the x-axis labels to avoid overlap

plt.xticks(rotation=45)

# Optionally, limit the number of x-ticks for better readability

plt.xticks(stock_data.index[::len(stock_data)//10])  # Adjust the step size based on your data size

# Show the plot

plt.show()

In [ ]:
# Assuming 'stock_data' has a 'Date' column or is using Date as the index

plt.figure(figsize=(10, 6))

plt.plot(stock_data['Open'], label='Open', color='red', linewidth=2)

plt.plot(stock_data['Close'], label='Close', color='green', linewidth=1)

# Adding labels and title

plt.title('Stock Open and Close Prices')

plt.xlabel('Date')

plt.ylabel('Price')

plt.legend()

# Rotating the x-axis labels to avoid overlap

plt.xticks(rotation=45)

# Optionally, limit the number of x-ticks for better readability

plt.xticks(stock_data.index[::len(stock_data)//10])  # Adjust the step size based on your data size

# Show the plot

plt.show()

In [ ]:
#Data Preprocessing

target_y = stock_data['Close']

X_feat = stock_data.iloc[:,0:3]

stock_data = stock_data[['Open', 'High', 'Low', 'Close']]

In [ ]:
#Feature Scaling

sc = StandardScaler()

stock_data_ft = sc.fit_transform(stock_data.values)

stock_data_ft = pd.DataFrame(columns=stock_data.columns,

                            data=stock_data_ft,

                            index=stock_data.index)


def lstm_split(data, n_steps):

    X, y = [], []

    for i in range(len(data) - n_steps):

        # Extract a window of size `n_steps` for features

        X.append(data[i:i + n_steps, :-1])  # All but last column

        # Extract the `n_steps`-th target value

        y.append(data[i + n_steps - 1, -1])  # Target value (last column)

    return np.array(X), np.array(y)


X1, y1 = lstm_split(stock_data_ft.values, n_steps=1)

train_split=0.8

split_idx= int(np.ceil(len(X1)*train_split))

date_index = stock_data_ft.index

X_train, X_test = X1[:split_idx], X1[split_idx:]

y_train, y_test = y1[:split_idx], y1[split_idx:]

X_train_date, X_test_date = date_index[:split_idx], date_index[split_idx:]

print(X1. shape, X_train. shape, X_test.shape, y_test.shape)



In [ ]:
# Define LSTM Model

lstm = Sequential()

# LSTM Layer (Set return_sequences=False if you want one output per sample)

lstm.add(LSTM(32, input_shape=(X_train.shape[1], X_train.shape[2]), activation='relu', return_sequences=True))

# Output Layer

lstm.add(Dense(1))

# Compile the Model

lstm.compile(loss='mean_squared_error', optimizer='adam')

# Print Model Summary

lstm.summary()

# Early Stopping Callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = lstm.fit(X_train, y_train, epochs=100, batch_size=4, validation_split=0.2, verbose=2, shuffle=False)



In [ ]:
# Plot the training loss

plt.figure(figsize=(10, 6))

plt.plot(history.history['loss'], label='Training Loss', color='blue', linewidth=2)

# Annotating the plot

plt.title('Training Loss vs. Epochs', fontsize=16)

plt.xlabel('Epochs', fontsize=14)

plt.ylabel('Loss', fontsize=14)

plt.legend(fontsize=12)

plt.grid(True)

plt.show()

In [ ]:
y_pred = lstm.predict(X_test)
#To evaluate, first, we plot the curve for true values and overlap it with that for the predicted values using the code below.

#If n = 1

import matplotlib.pyplot as plt

# Ensure y_test is squeezed (flattened) if it has an extra dimension

y_test_flat = y_test.squeeze()

y_pred_flat = y_pred.squeeze()

# Plot the true values and predictions

plt.figure(figsize=(12, 6))

plt.plot(y_test_flat, label="True Values", color="green", linewidth=2)

plt.plot(y_pred_flat, label="LSTM Predictions", color="red", linewidth=2)

# Add labels, title, and legend

plt.title("LSTM Predictions vs. True Values", fontsize=16)

plt.xlabel("Time Step", fontsize=14)

plt.ylabel("Value", fontsize=14)

plt.legend(fontsize=12)

plt.grid(True)

# Show the plot

plt.show()

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np
from statsmodels.tsa.api import SimpleExpSmoothing

mse = mean_squared_error(y_test, y_pred_flat)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(y_test, y_pred_flat)
print("RMSE: ", rmse)
print("MAPE: ", mape)

lstm = Sequential()
lstm.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2]),
              activation='relu', return_sequences=True))
lstm.add(LSTM(50, activation='relu', return_sequences=True))
lstm.add(LSTM(50, activation='relu'))
lstm.add(Dense(1))
lstm.compile(loss='mean_squared_error', optimizer='adam')
lstm.summary()

history = lstm.fit(X_train, y_train,
                   epochs=100, batch_size=4,
                   verbose=2, shuffle=False)
y_pred = lstm.predict(X_test)

plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss', color='blue', linewidth=2)
plt.title('Training Loss vs. Epochs', fontsize=16)
plt.xlabel('Epochs', fontsize=14)
plt.ylabel('Loss', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True)
plt.show()

import matplotlib.pyplot as plt
y_test_flat = y_test.squeeze()
plt.figure(figsize=(12, 6))
plt.plot(y_test_flat, label="True Values", color="green", linewidth=2)
plt.plot(y_pred, label="LSTM Predictions", color="red", linewidth=2)
plt.title("LSTM Predictions vs. True Values", fontsize=16)
plt.xlabel("Time Step", fontsize=14)
plt.ylabel("Value", fontsize=14)
plt.legend(fontsize=12)
plt.grid(True)
plt.show()

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(y_test, y_pred)
print("RMSE: ", rmse)
print("MAPE: ", mape)

n_steps = 10
X1, y1 = lstm_split(stock_data_ft.values, n_steps=n_steps)
train_split = 0.8
split_idx = int(np.ceil(len(X1) * train_split))
date_index = stock_data_ft.index
X_train, X_test = X1[:split_idx], X1[split_idx:]
y_train, y_test = y1[:split_idx], y1[split_idx:]
X_train_date, X_test_date = date_index[:split_idx], date_index[split_idx:-n_steps]
print(X1.shape, X_train.shape, X_test.shape, X_test_date.shape, y_test.shape)

lstm = Sequential()
lstm.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2]),
              activation='relu', return_sequences=True))
lstm.add(LSTM(50, activation='relu', return_sequences=True))
lstm.add(LSTM(50, activation='relu'))
lstm.add(Dense(1))
lstm.compile(loss='mean_squared_error', optimizer='adam')
lstm.summary()

history = lstm.fit(X_train, y_train,
                   epochs=100, batch_size=4,
                   verbose=2, shuffle=False)

plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss', color='blue', linewidth=2)
plt.title('Training Loss vs. Epochs', fontsize=16)
plt.xlabel('Epochs', fontsize=14)
plt.ylabel('Loss', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True)
plt.show()

y_pred = lstm.predict(X_test)

import matplotlib.pyplot as plt
y_test_flat = y_test.squeeze()
plt.figure(figsize=(12, 6))
plt.plot(y_test_flat, label="True Values", color="green", linewidth=2)
plt.plot(y_pred, label="LSTM Predictions", color="red", linewidth=2)
plt.title("LSTM Predictions vs. True Values", fontsize=16)
plt.xlabel("Time Step", fontsize=14)
plt.ylabel("Value", fontsize=14)
plt.legend(fontsize=12)
plt.grid(True)
plt.show()

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(y_test, y_pred)
print("RMSE: ", rmse)
print("MAPE: ", mape)

train_split = 0.8
split_idx = int(np.ceil(len(stock_data) * train_split))
train = stock_data[['Close']].iloc[:split_idx]
test = stock_data[['Close']].iloc[split_idx:]
test_pred = np.array([train.rolling(10).mean().iloc[-1]] * len(test)).reshape((-1, 1))
print('Test RMSE: %.3f' % np.sqrt(mean_squared_error(test, test_pred)))
print('Test MAPE: %.3f' % mean_absolute_percentage_error(test, test_pred))
plt.figure(figsize=(10, 5))
plt.plot(test)
plt.plot(test_pred)
plt.show()

X = stock_data[['Close']].values
train_split = 0.8
split_idx = int(np.ceil(len(X) * train_split))
train = X[:split_idx]
test = X[split_idx:]
test_concat = np.array([]).reshape((0, 1))
for i in range(len(test)):
    train_fit = np.concatenate((train, np.asarray(test_concat)))
    fit = SimpleExpSmoothing(np.asarray(train_fit)).fit(smoothing_level=0.1)
    test_pred = fit.forecast(1)
    test_concat = np.concatenate((np.asarray(test_concat), test_pred.reshape((-1, 1))))
print('Test RMSE: %.3f' % np.sqrt(mean_squared_error(test, test_concat)))
print('Test MAPE: %.3f' % mean_absolute_percentage_error(test, test_concat))
plt.figure(figsize=(10, 5))
plt.plot(test)
plt.plot(test_concat)
plt.show()